- 최소 길이: 358 characters
- 최대 길이: 3285 characters
- 평균 길이: 약 1227 characters
- 중앙값 길이: 1110 characters

In [1]:
from langchain_community.document_loaders.json_loader import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import os
import pandas as pd
import uuid

In [2]:
raw_data_path = './raw_docs/easystory.json'
output_path = './parse/'

In [3]:
def extract_json_metadata(record: dict, metadata: dict) -> dict:
    metadata["subject"] = record.get("subject")
    metadata["title"] = record.get("title")
    return metadata

In [4]:
def convert_to_corpus_with_langchain(raw_data_path: str, output_path: str) -> pd.DataFrame:
    documents = JSONLoader(
        file_path=raw_data_path,
        jq_schema=".data[]",  
        content_key="content",  
        text_content=False,
        metadata_func=extract_json_metadata
    ).load()
    
    documents = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128).split_documents(documents)
    
    rows = []
    for doc in documents:
        row = {
            "doc_id": str(uuid.uuid4()),  # 고유 ID 생성
            "texts": doc.page_content,
            "metadata": {
                "subject": doc.metadata.get("subject"),
                "title": doc.metadata.get("title")
            }
        }
        rows.append(row)
    
    df = pd.DataFrame(rows)
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    df.to_parquet(f"{output_path}/corpus.parquet", index=False)
    
    return df

In [5]:
convert_to_corpus_with_langchain(raw_data_path, output_path)

,doc_id,texts,metadata
0,34a527e1-3dfa-45db-a7b5-48de210fdb07,대부분의 사람들은 행복하게 살기를 원합니다. 행복한 삶을 이루기 위해 서는 건강한 ...,"{'subject': '1-1. 왜 경제이해력이 중요한가? ', 'title': '..."
1,18c6c9c1-4791-4a1a-89e2-e8bde2691f6f,경제상식이나 지식이 우리를 한 순간에 부자로 만들어 주지는 않습니다. 그렇지만 우리...,"{'subject': '1-1. 왜 경제이해력이 중요한가? ', 'title': '..."
2,ef94032a-37e8-4707-8c50-5e04383b7785,이러한 경제상식과 지식이 세계경제와 나라경제에서 여러 가지 경제현 상의 흐름과 실태...,"{'subject': '1-1. 왜 경제이해력이 중요한가? ', 'title': '..."
3,b43af607-c9a0-4149-a0b5-996db5087274,"일상생활은 경제활동의 연속입니다. 일반적으로 옷과 음식, 냉장고와 자동차 같은 재화...","{'subject': '1-2. 생활은 선택의 연속 ', 'title': '경제활동..."
4,5b6cba7a-b36a-42ee-a188-d3946d2a5c2e,경제문제는 이렇게 이용 가능한 자원이 희소하기 때문에 생깁니다. 즉 사람들이 각자 ...,"{'subject': '1-2. 생활은 선택의 연속 ', 'title': '경제활동..."
...,...,...,...
236,cf2c440c-3c18-4c38-bf6c-1a39efcaf8f3,노동절약적 기술진보의 진행에도 불구하고 일자리가 뚜렷히 줄어들지 않은 이유는 크게 ...,"{'subject': '15-3. 4차 산업혁명과 일자리 ', 'title': '기..."
237,5324757d-49fd-46cb-84be-282ed2d11dc9,여기서 우려되는 점은 기술변화 및 이로 인한 산업구조 재편 과정에서 중간 일자리가 ...,"{'subject': '15-3. 4차 산업혁명과 일자리 ', 'title': '일..."
238,ea068563-ae22-4885-b0a5-3ab529339d16,한편 인공지능의 도입이 기존 자동화의 한계를 뛰어넘어 비정형적 노동 역시 자동화가 ...,"{'subject': '15-3. 4차 산업혁명과 일자리 ', 'title': '일..."
239,78890c35-062b-4e3f-b051-7ebbc7f2e95b,그렇지만 인공지능 기술이 인간과 대등한 수준의 유연성을 가지고 비정형적 노동을 수행...,"{'subject': '15-3. 4차 산업혁명과 일자리 ', 'title': '일..."


In [6]:
check = pd.read_parquet('./parse/corpus.parquet', engine='pyarrow')
check.head(10)

,doc_id,texts,metadata
0,34a527e1-3dfa-45db-a7b5-48de210fdb07,대부분의 사람들은 행복하게 살기를 원합니다. 행복한 삶을 이루기 위해 서는 건강한 ...,"{'subject': '1-1. 왜 경제이해력이 중요한가? ', 'title': '..."
1,18c6c9c1-4791-4a1a-89e2-e8bde2691f6f,경제상식이나 지식이 우리를 한 순간에 부자로 만들어 주지는 않습니다. 그렇지만 우리...,"{'subject': '1-1. 왜 경제이해력이 중요한가? ', 'title': '..."
2,ef94032a-37e8-4707-8c50-5e04383b7785,이러한 경제상식과 지식이 세계경제와 나라경제에서 여러 가지 경제현 상의 흐름과 실태...,"{'subject': '1-1. 왜 경제이해력이 중요한가? ', 'title': '..."
3,b43af607-c9a0-4149-a0b5-996db5087274,"일상생활은 경제활동의 연속입니다. 일반적으로 옷과 음식, 냉장고와 자동차 같은 재화...","{'subject': '1-2. 생활은 선택의 연속 ', 'title': '경제활동..."
4,5b6cba7a-b36a-42ee-a188-d3946d2a5c2e,경제문제는 이렇게 이용 가능한 자원이 희소하기 때문에 생깁니다. 즉 사람들이 각자 ...,"{'subject': '1-2. 생활은 선택의 연속 ', 'title': '경제활동..."
5,336985db-68f3-4560-9b8a-f52d1d40d26d,"우리는 경제생활에서 희소성 때문에 선택을 하며 살고 있는데, 모든 선택에는 항상 무...","{'subject': '1-2. 생활은 선택의 연속 ', 'title': '선택과 ..."
6,3a481bdc-de23-4680-86b0-acd51abc6ad7,모든 경제적 결정에는 대가를 지급해야 합니다. 따라서 최선의 선택을 하려면 각각의 ...,"{'subject': '1-2. 생활은 선택의 연속 ', 'title': '하나를 ..."
7,8537d533-1794-4c9b-b937-c7b0570018ed,앞에서 우리는 희소성 때문에 발생하는 경제문제를 해소하기 위하여 편익과 비용을 비교...,"{'subject': '1-3. 경제공부는 기본개념의 이해에서 시작 ', 'titl..."
8,a42130a8-fd25-4ba0-8000-a1c1c199abae,위의 사례에서와 같이 선택에 따르는 추가적인 행복과 추가적인 희생을 비교하여 선택하...,"{'subject': '1-3. 경제공부는 기본개념의 이해에서 시작 ', 'titl..."
9,63a3a8d2-574e-497e-83f9-61e54e687f26,"한편 경제주체들이 편익과 비용 개념을 한계적으로 고려하여 의사결정을 할 때, 주변 ...","{'subject': '1-3. 경제공부는 기본개념의 이해에서 시작 ', 'titl..."
